In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import pdb
from tqdm import tqdm
from collections import OrderedDict
import copy

import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.insert(0, './src')
from data import Dataset
from models import HMC_vanilla

In [2]:
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'

In [3]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [4]:
args = dotdict({})

args['device'] = device
args['torchType'] = torch.float32
args['dataset_name'] = 'boston_housing'
if args['dataset_name'].find('mnist') > -1:
    args['num_epoches'] = 201
    args['print_info'] = 50
    args['n_IS'] = 10000
    
    args['train_batch_size'] = 100
    args['val_dataset'] = 10000
    args['val_batch_size'] = 100
    args['test_batch_size'] = 100
else:
    args['n_IS'] = 1000
    args['num_epoches'] = 10001
    args['print_info'] = 1000
    args['train_batch_size'] = 100
    args['val_dataset'] = 100
    args['val_batch_size'] = 20
    args['test_batch_size'] = 10

In [5]:
dataset = Dataset(args)
args.in_features = dataset.in_features[0]
dataset.in_features[0]

/home/nkotelevskii/anaconda3/envs/condatorch/lib/python3.7/site-packages/alpaca/dataloader/boston_housing.py:37: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  self.df = pd.read_table(data_path, names=self.column_names, header=None, delim_whitespace=True)


Train data shape 406


13

In [6]:
X_train = torch.tensor(dataset.x_train, device=device, dtype=torch.float32)
y_train = torch.tensor(dataset.y_train, device=device, dtype=torch.float32)

X_val = torch.tensor(dataset.x_val, device=device, dtype=torch.float32)
y_val = torch.tensor(dataset.y_val, device=device, dtype=torch.float32)

In [7]:
def load_parameters(model, param_vector, params_shapes):
    left_boader = 0
    tensors = []
    for shape in params_shapes:
        cur_len = np.prod(shape)
        tensors.append(param_vector[left_boader: left_boader+cur_len].view(*shape))
        left_boader += cur_len
    d = OrderedDict()
    model_clone = copy.deepcopy(model)
    for i, name in enumerate(model_clone.named_parameters()):
        d[name[0]] = tensors[i]
    model_clone.load_state_dict(d)
    model_clone.to(model.device)
    return model_clone

In [8]:
class Simple_model(nn.Module):
    def __init__(self, args):
        super(Simple_model, self).__init__()
        in_features = args.in_features
        self.device = args.device
        self.l1 = nn.Linear(in_features, 3*in_features)
        self.l2 = nn.Linear(3*in_features, 1)
        self.activation = nn.Softplus()
        self.std_normal = torch.distributions.Normal(loc=torch.tensor(0., dtype=torch.float32, device=self.device),
                                                     scale=torch.tensor(1., dtype=torch.float32, device=self.device))
        
    def forward(self, x):
        h = self.activation(self.l1(x))
        h = self.l2(h)
        return h
    
    def get_logdensity(self, x, z):
        '''
        x - data vector
        z - vector of all parameters
        '''
#         pdb.set_trace()
        log_prior = self.std_normal.log_prob(z).sum()
        loaded_model = load_parameters(self, z, params_shapes)
        y_pred = loaded_model(x)
        log_likelihood = torch.distributions.Normal(loc=y_train,
                                                     scale=torch.tensor(1., dtype=torch.float32, device=self.device)).log_prob(y_pred).sum()
#         print('log_likelihood', log_likelihood)
        logdensity = log_likelihood + log_prior
        return logdensity

In [9]:
model = Simple_model(args)

In [10]:
params_shapes = [p.shape for p in list(model.parameters())]
print(params_shapes)
overall_params = sum([np.prod(el) for el in params_shapes])
print(f'Overall number of parameters is {overall_params}')

[torch.Size([39, 13]), torch.Size([39]), torch.Size([1, 39]), torch.Size([1])]
Overall number of parameters is 586


In [11]:
initial_parameters = torch.randn((overall_params, 1), dtype=torch.float32, device=device)

In [12]:
L = 100000 # num of hmc transitions
args.N = 3
args.alpha = 0.5
args.gamma = 0.001
args.use_partialref = False
args.use_barker = False


hmc_transition = HMC_vanilla(args)   

In [13]:
q_old = initial_parameters
q_new = q_old
a_list = []

model_new = copy.deepcopy(load_parameters(model, q_old, params_shapes))

for l in tqdm(range(L)):
    q_new, _, _, _, a = hmc_transition.make_transition(q_old=q_new, p_old=None, target_distr=model_new, x=X_train)
    a_list.append(a.cpu().numpy())
    model_new = copy.deepcopy(load_parameters(model_new, q_new, params_shapes))
    if l % 500 == 0:
        print(f'Mean acceptance rate {np.mean(a_list)}')
        a_list = []
        y_pred = model_new(X_train)
        print(f"At L={l} MSE on train is {((y_pred.squeeze() - y_train.squeeze())**2).mean()}")
        
        y_pred = model_new(X_val)
        print(f"At L={l} MSE on validation is {((y_pred.squeeze() - y_val.squeeze())**2).mean()}")

  0%|          | 1/100000 [00:00<4:34:02,  6.08it/s]

Mean acceptance rate 0.0
At L=0 MSE on train is 5095579.0
At L=0 MSE on validation is 5332677.5


  1%|          | 520/100000 [00:04<13:43, 120.86it/s]

Mean acceptance rate 0.47200000286102295
At L=500 MSE on train is 321977.21875
At L=500 MSE on validation is 259424.3125


  1%|          | 1014/100000 [00:08<13:38, 120.88it/s]

...

 99%|█████████▉| 99022/100000 [13:25<00:08, 120.40it/s]

Mean acceptance rate 0.0
At L=99000 MSE on train is 123.0765609741211
At L=99000 MSE on validation is 186.82516479492188


100%|█████████▉| 99516/100000 [13:29<00:04, 119.30it/s]

Mean acceptance rate 0.0
At L=99500 MSE on train is 123.0765609741211
At L=99500 MSE on validation is 186.82516479492188


100%|██████████| 100000/100000 [13:33<00:00, 122.91it/s]
